In [3]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 975.5 kB/s eta 0:00:00a 0:00:01


In [4]:
from time import sleep
import pandas as pd
import numpy as np
import warnings  
import random
import uuid
import sys
import os
os.listdir('/kaggle/input/')
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', 500) 
pd.set_option('display.max_rows', None) 
pd.set_option('display.float_format', lambda x: '%.3f' % x) 
warnings.simplefilter(action="ignore")
df=pd.read_excel('../input/faculty-student-data/student_info.xlsx')
df.sort_index().reset_index(drop = True)
lessons={'Data Science':'Not Appointed','Artificial Intelligence':'Demis Hassabis','Virtual Reality':'Morton Heilig','Games Development':'Oles Shishkovtsov','3D Graphics And Animation':'John Lasseter','Mobile Development':'Xavier Ducrohet','Interaction Design':'Viacheslav Aristov','Natural Language Processing':'Christopher D. Manning','Intelligent Signal Processing':'Claude Shannon'}

In [5]:
def load_screen():
  print("Your action will result in a system restart or shutdown.\n")
  for i in range(101):
    sys.stdout.write('\r')
    sys.stdout.write("[%-10s] %d%%" % ('='*i, 1*i))
    sys.stdout.flush()
    sleep(0.02)

In [6]:
def create_teacher(profession):
  profession=profession.title()
  if lessons[profession]!="Not Appointed":
   print("Dear related, there is not yet a vacancy for this specialty in our faculty. Please try again in the future.")
  elif list(lessons.keys()).count(profession) == 0:
   print("Dear related, your area of specialization is not an area of specialization to be found in the lessons of the students in our faculty. You can apply again in the future.")
  else:
   lessons[profession]=input("Enter the name of the teacher:").strip().title()+" "+input("Enter the surname of the teacher:").strip().title()
   for i in range(len(df)):
    if df.loc[i]['lesson_name']==profession:
     df.at[i,'lessons_teacher']=lessons[profession]
   print("Your transaction has been successfully completed.")
   display(df[df['lesson_name']==profession])

def delete_teacher(profession):
  if list(lessons.keys()).count(profession.title()) != 0:
    lessons[profession]="Not Appointed"
    for i in range(len(df)):
      if df.loc[i]['lesson_name']==profession:
        df.at[i,'lessons_teacher']=lessons[profession] 
    display(pd.DataFrame({'Lessons':lessons.keys(), 'Faculty Members':lessons.values()}))
    display(df[df['lesson_name']==profession])
    print("Your transaction has been successfully completed.")
  else:
    load_screen()

def create_student(student_name,student_surname):
  id=int(str(uuid.uuid4().fields[-1])[:5])
  if list(df.drop_duplicates(subset=['student_id'])['student_id']).count(id) == 1:
    id=int(str(uuid.uuid4().fields[-1])[:5])
  for i in lessons.keys():
   df.loc[len(df)] = [id,
             student_name.strip().title(),
             student_surname.strip().title(),i,lessons[i],"Not Calculated","Not Calculated","Not Calculated"]
  display(df[df['student_id']==id])
  print("Your transaction has been successfully completed.")

def delete_student(student_id):
  if len(df[df['student_id']==int(student_id)])!=0:
    df.drop(df[df['student_id']==student_id].index,axis=0,inplace=True)
    display(df.drop_duplicates(subset=['student_id'])[['student_id',"student_name","student_surname"]])
    print("Your transaction has been successfully completed.")
  else:
    load_screen()

def create_lesson(lesson_name):
 if list(lessons.keys()).count(lesson_name.title()) == 0:    
  lessons.update({lesson_name.title():"Not appointed"})
  for i in list(df[df['lesson_name']==list(lessons.keys())[-2]].index):
    c=list(df.iloc[i])
    df.loc[i] = list([c[0],c[1],c[2],list(lessons.keys())[-1],list(lessons.values())[-1],"Not Calculated","Not Calculated","Not Calculated"])
  display(pd.DataFrame({'Lessons':lessons.keys()}))
  display(df[df['lesson_name']==lesson_name.title()])
  print("Your transaction has been successfully completed.")
 else:
   load_screen()

def delete_lesson(lesson_name):
  if list(lessons.keys()).count(lesson_name.title()) == 1:
    del lessons[lesson_name.title()]
    df.drop(df[(df['lesson_name']==lesson_name)].index,axis=0,inplace=True)
    display(pd.DataFrame({'Lessons':lessons.keys()}))
    print("Your transaction has been successfully completed.")
  else:
    load_screen()

In [7]:
def admin_processes():
  staff_choice=int(input("""*********WELCOME TO THE ADMIN PANEL*********

  '0' to create teacher registration
  '1' for teacher dismissal
  '2' to create a student record
  '3' for student expulsion
  '4' to open lesson
  '5' to close lesson\nYour Choice:"""))
  if staff_choice == 0:
    display(pd.DataFrame({'Lessons':lessons.keys(), 'Faculty Members':lessons.values()}))
    create_teacher(input("Enter the teacher's area of specialization:"))

  elif staff_choice == 1:
    display(pd.DataFrame({'Lessons':lessons.keys(), 'Faculty Members':lessons.values()}))
    delete_teacher(input("Enter the lesson for which the faculty member you want to dismiss is responsible:"))
    
  elif staff_choice == 2:
    create_student(input("Enter the student name:").strip().title(),input("Enter the student surname:").strip().title())

  elif staff_choice == 3:
    display(df.drop_duplicates(subset=['student_id'])[['student_id',"student_name","student_surname"]])
    delete_student(int(input("Enter the id of the student you want to expel:")))

  elif staff_choice == 4:
    create_lesson(input("Enter the name of the lesson you want to open:"))

  elif staff_choice == 5:
    display(pd.DataFrame({'Lessons':lessons.keys()}))
    delete_lesson(input("Enter the name of the lesson you want to close:"))
  else:
    load_screen()
  if int(input("Would you like to take any other action?\nClick '0' if you want to take another action and '1' if you don't\nYour Choice:")) == 0:
    admin_processes()
  else:
    load_screen()

In [8]:
def enter_note(lesson): 
  if len(df[(df["lesson_name"]==lesson) & (df["number_grade"]=="Not Calculated") & (df["lessons_teacher"]!="Not Appointed")])!=0:
    for i in df[(df['lesson_name']==lesson) & (df['number_grade']=="Not Calculated") & (df["lessons_teacher"]!="Not Appointed")].index:
      note=int(input(df.loc[i][:-3].to_string()+" enter the grade of this student:"))
      if note >= 0 and note <= 100:
        if note >= 90 and note <= 100:
          df.at[i,['number_grade','letter_grade','lesson_status']]=[note,"AA","Pass"]

        elif note >=85 and note <= 89:
          df.at[i,['number_grade','letter_grade','lesson_status']]=[note,"BA","Pass"]

        elif note >= 80 and note <= 84:
          df.at[i,['number_grade','letter_grade','lesson_status']]=[note,"BB","Pass"]

        elif note >= 75 and note <= 79:
          df.at[i,['number_grade','letter_grade','lesson_status']]=[note,"CB","Pass"]

        elif note >= 70 and note <= 74:
          df.at[i,['number_grade','letter_grade','lesson_status']]=[note,"CC","Pass"]

        elif note >= 65 and note <= 69:
          df.at[i,['number_grade','letter_grade','lesson_status']]=[note,"DC",'Fail']

        elif note >= 60 and note <= 64:
          df.at[i,['number_grade','letter_grade','lesson_status']]=[note,"DD",'Fail']

        elif note >= 50 and note <= 59:
          df.at[i,['number_grade','letter_grade','lesson_status']]=[note,"FD",'Fail']
          
        elif note <= 49:
          df.at[i,['number_grade','letter_grade','lesson_status']]=[note,"FF",'Fail']

    print("Your transactions have been successfully completed.")
    display(df[df['lesson_name']==lesson])
  else:
    print("There are no unentered grades.")

def update_note(lesson):
  if len(df[(df["lesson_name"]==lesson) & (df["number_grade"]!="Not Calculated") & (df["lessons_teacher"]!="Not Appointed")])!=0:
    display(df[(df['lesson_name']==lesson) & (df['number_grade']!="Not Calculated")])
    id=int(input("Enter the ID of the student whose grade you want to correct:"))
    note=int(input("Enter the student's new grade:"))
    if note >= 90 and note <= 100:
      df.at[df[(df['student_id']==id) & (df['lesson_name']==lesson)].index,['number_grade','letter_grade','lesson_status']]=[note,"AA","Pass"]

    elif note >=85 and note <= 89:
      df.at[df[(df['student_id']==id) & (df['lesson_name']==lesson)].index,['number_grade','letter_grade','lesson_status']]=[note,"BA","Pass"]

    elif note >= 80 and note <= 84:
      df.at[df[(df['student_id']==id) & (df['lesson_name']==lesson)].index,['number_grade','letter_grade','lesson_status']]=[note,"BB","Pass"]

    elif note >= 75 and note <= 79:
      df.at[df[(df['student_id']==id) & (df['lesson_name']==lesson)].index,['number_grade','letter_grade','lesson_status']]=[note,"CB","Pass"]

    elif note >= 70 and note <= 74:
      df.at[df[(df['student_id']==id) & (df['lesson_name']==lesson)].index,['number_grade','letter_grade','lesson_status']]=[note,"CC","Pass"]

    elif note >= 65 and note <= 69:
      df.at[df[(df['student_id']==id) & (df['lesson_name']==lesson)].index,['number_grade','letter_grade','lesson_status']]=[note,"DC",'Fail']

    elif note >= 60 and note <= 64:
      df.at[df[(df['student_id']==id) & (df['lesson_name']==lesson)].index,['number_grade','letter_grade','lesson_status']]=[note,"DD",'Fail']

    elif note >= 50 and note <= 59:
      df.at[df[(df['student_id']==id) & (df['lesson_name']==lesson)].index,['number_grade','letter_grade','lesson_status']]=[note,"FD",'Fail']
      
    elif note <= 49:
      df.at[df[(df['student_id']==id) & (df['lesson_name']==lesson)].index,['number_grade','letter_grade','lesson_status']]=[note,"FF",'Fail']
    display(df[(df['lesson_name']==lesson) & (df['student_id']==id)])
    print("Your transaction has been successfully completed.")


def staff_processes():
  print("*********WELCOME TO THE MANAGEMENT INFORMATION SYSTEM*********")
  staff_choice=int(input("'0' to enter a grade\n'1' to correct a grade\nYour Choice:"))
    
  if staff_choice == 0:
    display(pd.DataFrame({'Lessons':lessons.keys()}))
    enter_note(input("Enter the lessons you are responsible for:"))
    if int(input("Would you like to take any other action?\nClick '0' if you want to take another action and '1' if you don't\nYour Choice:")) == 0:
      staff_processes()
    
    else:
      load_screen()

  elif staff_choice == 1:
    display(pd.DataFrame({'Lessons':lessons.keys()}))
    update_note(input("Enter the lessons you are responsible for:"))
    if int(input("Would you like to take any other action?\nClick '0' if you want to take another action and '1' if you don't\nYour Choice:")) == 0:
      staff_processes()
    
    else:
      load_screen()
    
  else:
    load_screen()

In [9]:
def enter_system():
  print("""*********WELCOME TO THE FACULTY SYSTEM OF COMPUTER ENGINEERING*********""")
  system_choice=int(input("'0' to log in to the Management Information System\n'1' to log in to the Student Information System\nYour Choice:"))
  if system_choice == 0:
    staff_processes()
  elif system_choice == 1:
    student_id=int(input("Enter your ID to view your grades:"))
    if len(df[df['student_id']==student_id])!=0:
        display(df[df['student_id']==student_id])
    else:
        load_screen()
  elif system_choice == 9:
    admin_processes()
  else:
    load_screen()

# I had to use the methods one by one because inputs was not visible in the output. If you want to run the code yourself just type enter_system()

# **Admin Processes**

In [51]:
display(pd.DataFrame({'Lessons':lessons.keys(), 'Faculty Members':lessons.values()}))
create_teacher(input("Enter the teacher's area of specialization:"))

,Lessons,Faculty Members
0,Data Science,Not Appointed
1,Artificial Intelligence,Demis Hassabis
2,Virtual Reality,Morton Heilig
3,Games Development,Oles Shishkovtsov
4,3D Graphics And Animation,John Lasseter
5,Mobile Development,Xavier Ducrohet
6,Interaction Design,Viacheslav Aristov
7,Natural Language Processing,Christopher D. Manning
8,Intelligent Signal Processing,Claude Shannon


Enter the teacher's area of specialization: Data Science
Enter the name of the teacher: Yann
Enter the surname of the teacher: LeCun


Your transaction has been successfully completed.


,student_id,student_name,student_surname,lesson_name,lessons_teacher,number_grade,letter_grade,lesson_status
0,14867,Johnson,Williams,Data Science,Yann Lecun,Not Calculated,Not Calculated,Not Calculated
9,10880,Elizabeth,Brown,Data Science,Yann Lecun,Not Calculated,Not Calculated,Not Calculated
18,85347,Julia,Toff,Data Science,Yann Lecun,Not Calculated,Not Calculated,Not Calculated
27,27166,Eren,Cice,Data Science,Yann Lecun,Not Calculated,Not Calculated,Not Calculated


In [52]:
display(pd.DataFrame({'Lessons':lessons.keys(), 'Faculty Members':lessons.values()}))
delete_teacher(input("Enter the lesson for which the faculty member you want to dismiss is responsible:"))

,Lessons,Faculty Members
0,Data Science,Yann Lecun
1,Artificial Intelligence,Demis Hassabis
2,Virtual Reality,Morton Heilig
3,Games Development,Oles Shishkovtsov
4,3D Graphics And Animation,John Lasseter
5,Mobile Development,Xavier Ducrohet
6,Interaction Design,Viacheslav Aristov
7,Natural Language Processing,Christopher D. Manning
8,Intelligent Signal Processing,Claude Shannon


Enter the lesson for which the faculty member you want to dismiss is responsible: Data Science


,Lessons,Faculty Members
0,Data Science,Not Appointed
1,Artificial Intelligence,Demis Hassabis
2,Virtual Reality,Morton Heilig
3,Games Development,Oles Shishkovtsov
4,3D Graphics And Animation,John Lasseter
5,Mobile Development,Xavier Ducrohet
6,Interaction Design,Viacheslav Aristov
7,Natural Language Processing,Christopher D. Manning
8,Intelligent Signal Processing,Claude Shannon


,student_id,student_name,student_surname,lesson_name,lessons_teacher,number_grade,letter_grade,lesson_status
0,14867,Johnson,Williams,Data Science,Not Appointed,Not Calculated,Not Calculated,Not Calculated
9,10880,Elizabeth,Brown,Data Science,Not Appointed,Not Calculated,Not Calculated,Not Calculated
18,85347,Julia,Toff,Data Science,Not Appointed,Not Calculated,Not Calculated,Not Calculated
27,27166,Eren,Cice,Data Science,Not Appointed,Not Calculated,Not Calculated,Not Calculated


Your transaction has been successfully completed.


In [53]:
create_student(input("Enter the student name:").strip().title(),input("Enter the student surname:").strip().title())

Enter the student name: Jones
Enter the student surname: Miller


,student_id,student_name,student_surname,lesson_name,lessons_teacher,number_grade,letter_grade,lesson_status
36,26152,Jones,Miller,Data Science,Not Appointed,Not Calculated,Not Calculated,Not Calculated
37,26152,Jones,Miller,Artificial Intelligence,Demis Hassabis,Not Calculated,Not Calculated,Not Calculated
38,26152,Jones,Miller,Virtual Reality,Morton Heilig,Not Calculated,Not Calculated,Not Calculated
39,26152,Jones,Miller,Games Development,Oles Shishkovtsov,Not Calculated,Not Calculated,Not Calculated
40,26152,Jones,Miller,3D Graphics And Animation,John Lasseter,Not Calculated,Not Calculated,Not Calculated
41,26152,Jones,Miller,Mobile Development,Xavier Ducrohet,Not Calculated,Not Calculated,Not Calculated
42,26152,Jones,Miller,Interaction Design,Viacheslav Aristov,Not Calculated,Not Calculated,Not Calculated
43,26152,Jones,Miller,Natural Language Processing,Christopher D. Manning,Not Calculated,Not Calculated,Not Calculated
44,26152,Jones,Miller,Intelligent Signal Processing,Claude Shannon,Not Calculated,Not Calculated,Not Calculated


Your transaction has been successfully completed.


In [54]:
display(df.drop_duplicates(subset=['student_id'])[['student_id',"student_name","student_surname"]])
delete_student(int(input("Enter the id of the student you want to expel:")))

,student_id,student_name,student_surname
0,14867,Johnson,Williams
9,10880,Elizabeth,Brown
18,85347,Julia,Toff
27,27166,Eren,Cice
36,26152,Jones,Miller


Enter the id of the student you want to expel: 26152


,student_id,student_name,student_surname
0,14867,Johnson,Williams
9,10880,Elizabeth,Brown
18,85347,Julia,Toff
27,27166,Eren,Cice


Your transaction has been successfully completed.


In [55]:
create_lesson(input("Enter the name of the lesson you want to open:"))

Enter the name of the lesson you want to open: Augmented Reality


,Lessons
0,Data Science
1,Artificial Intelligence
2,Virtual Reality
3,Games Development
4,3D Graphics And Animation
5,Mobile Development
6,Interaction Design
7,Natural Language Processing
8,Intelligent Signal Processing
9,Augmented Reality


,student_id,student_name,student_surname,lesson_name,lessons_teacher,number_grade,letter_grade,lesson_status
8,14867,Johnson,Williams,Augmented Reality,Not appointed,Not Calculated,Not Calculated,Not Calculated
17,10880,Elizabeth,Brown,Augmented Reality,Not appointed,Not Calculated,Not Calculated,Not Calculated
26,85347,Julia,Toff,Augmented Reality,Not appointed,Not Calculated,Not Calculated,Not Calculated
35,27166,Eren,Cice,Augmented Reality,Not appointed,Not Calculated,Not Calculated,Not Calculated


Your transaction has been successfully completed.


In [56]:
display(pd.DataFrame({'Lessons':lessons.keys()}))
delete_lesson(input("Enter the name of the lesson you want to close:"))

,Lessons
0,Data Science
1,Artificial Intelligence
2,Virtual Reality
3,Games Development
4,3D Graphics And Animation
5,Mobile Development
6,Interaction Design
7,Natural Language Processing
8,Intelligent Signal Processing
9,Augmented Reality


Enter the name of the lesson you want to close: Augmented Reality


,Lessons
0,Data Science
1,Artificial Intelligence
2,Virtual Reality
3,Games Development
4,3D Graphics And Animation
5,Mobile Development
6,Interaction Design
7,Natural Language Processing
8,Intelligent Signal Processing


Your transaction has been successfully completed.


# **Teacher Processes**

In [58]:
display(pd.DataFrame({'Lessons':lessons.keys()}))
enter_note(input("Enter the lessons you are responsible for:"))

,Lessons
0,Data Science
1,Artificial Intelligence
2,Virtual Reality
3,Games Development
4,3D Graphics And Animation
5,Mobile Development
6,Interaction Design
7,Natural Language Processing
8,Intelligent Signal Processing


Enter the lessons you are responsible for: Artificial Intelligence
student_id                           14867
student_name                       Johnson
student_surname                   Williams
lesson_name        Artificial Intelligence
lessons_teacher             Demis Hassabis enter the grade of this student: 70
student_id                           10880
student_name                     Elizabeth
student_surname                      Brown
lesson_name        Artificial Intelligence
lessons_teacher             Demis Hassabis enter the grade of this student: 80
student_id                           85347
student_name                         Julia
student_surname                       Toff
lesson_name        Artificial Intelligence
lessons_teacher             Demis Hassabis enter the grade of this student: 80
student_id                           27166
student_name                          Eren
student_surname                       Cice
lesson_name        Artificial Intelligence
lessons_

Your transactions have been successfully completed.


,student_id,student_name,student_surname,lesson_name,lessons_teacher,number_grade,letter_grade,lesson_status
1,14867,Johnson,Williams,Artificial Intelligence,Demis Hassabis,70,CC,Pass
10,10880,Elizabeth,Brown,Artificial Intelligence,Demis Hassabis,80,BB,Pass
19,85347,Julia,Toff,Artificial Intelligence,Demis Hassabis,80,BB,Pass
28,27166,Eren,Cice,Artificial Intelligence,Demis Hassabis,15,FF,Fail


In [59]:
display(pd.DataFrame({'Lessons':lessons.keys()}))
update_note(input("Enter the lessons you are responsible for:"))

,Lessons
0,Data Science
1,Artificial Intelligence
2,Virtual Reality
3,Games Development
4,3D Graphics And Animation
5,Mobile Development
6,Interaction Design
7,Natural Language Processing
8,Intelligent Signal Processing


Enter the lessons you are responsible for: Artificial Intelligence


,student_id,student_name,student_surname,lesson_name,lessons_teacher,number_grade,letter_grade,lesson_status
1,14867,Johnson,Williams,Artificial Intelligence,Demis Hassabis,70,CC,Pass
10,10880,Elizabeth,Brown,Artificial Intelligence,Demis Hassabis,80,BB,Pass
19,85347,Julia,Toff,Artificial Intelligence,Demis Hassabis,80,BB,Pass
28,27166,Eren,Cice,Artificial Intelligence,Demis Hassabis,15,FF,Fail


Enter the ID of the student whose grade you want to correct: 27166
Enter the student's new grade: 100


,student_id,student_name,student_surname,lesson_name,lessons_teacher,number_grade,letter_grade,lesson_status
28,27166,Eren,Cice,Artificial Intelligence,Demis Hassabis,100,AA,Pass


Your transaction has been successfully completed.


# **Student Process**

In [60]:
student_id=int(input("Enter your ID to view your grades:"))
if len(df[df['student_id']==student_id])!=0:
  display(df[df['student_id']==student_id])
else:
  load_screen()

Enter your ID to view your grades: 27166


,student_id,student_name,student_surname,lesson_name,lessons_teacher,number_grade,letter_grade,lesson_status
27,27166,Eren,Cice,Data Science,Not Appointed,Not Calculated,Not Calculated,Not Calculated
28,27166,Eren,Cice,Artificial Intelligence,Demis Hassabis,100,AA,Pass
29,27166,Eren,Cice,Virtual Reality,Morton Heilig,Not Calculated,Not Calculated,Not Calculated
30,27166,Eren,Cice,Games Development,Oles Shishkovtsov,Not Calculated,Not Calculated,Not Calculated
31,27166,Eren,Cice,3D Graphics And Animation,John Lasseter,Not Calculated,Not Calculated,Not Calculated
32,27166,Eren,Cice,Mobile Development,Xavier Ducrohet,Not Calculated,Not Calculated,Not Calculated
33,27166,Eren,Cice,Interaction Design,Viacheslav Aristov,Not Calculated,Not Calculated,Not Calculated
34,27166,Eren,Cice,Natural Language Processing,Christopher D. Manning,Not Calculated,Not Calculated,Not Calculated
